In [1]:
import sys
sys.path.append("..")
sys.path.append("../models")

from models import ResNetSimCLR
import torch 

model = ResNetSimCLR(size=50, device="cuda", embedding_dim=64)
checkpoint = torch.load("../checkpoints/checkpoint_epoch_20.pt", map_location="cuda")

model.load_state_dict(checkpoint["model_state_dict"])

# Freeze all parameters in ResNetSimCLR
for param in model.parameters():
    param.requires_grad = False

GPU is being utilized


In [2]:
import torch
import torch.nn as nn

class ClassifierHead(nn.Module):
    def __init__(self, base_model):
        super(ClassifierHead, self).__init__()
        self.encoder = base_model  # Use the existing model as the encoder
        # Add new layers on top of the encoder
        self.fc1 = nn.Linear(base_model.embedding_dim, 64)  # First FC layer
        self.relu = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(64, 15)  # Classification head for 10 classes
        self.softmax = nn.Softmax(dim=1)  # Softmax for probabilities
    
    def forward(self, x):
        # Forward pass through the encoder
        x = self.encoder(x)
        # Pass through the new fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)  # Apply softmax for probabilities
        return x

# Instantiate the custom model
custom_model = ClassifierHead(model)

# Verify trainable parameters
for name, param in custom_model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

encoder.model.conv1.weight: Frozen
encoder.model.bn1.weight: Frozen
encoder.model.bn1.bias: Frozen
encoder.model.layer1.0.conv1.weight: Frozen
encoder.model.layer1.0.bn1.weight: Frozen
encoder.model.layer1.0.bn1.bias: Frozen
encoder.model.layer1.0.conv2.weight: Frozen
encoder.model.layer1.0.bn2.weight: Frozen
encoder.model.layer1.0.bn2.bias: Frozen
encoder.model.layer1.0.conv3.weight: Frozen
encoder.model.layer1.0.bn3.weight: Frozen
encoder.model.layer1.0.bn3.bias: Frozen
encoder.model.layer1.0.downsample.0.weight: Frozen
encoder.model.layer1.0.downsample.1.weight: Frozen
encoder.model.layer1.0.downsample.1.bias: Frozen
encoder.model.layer1.1.conv1.weight: Frozen
encoder.model.layer1.1.bn1.weight: Frozen
encoder.model.layer1.1.bn1.bias: Frozen
encoder.model.layer1.1.conv2.weight: Frozen
encoder.model.layer1.1.bn2.weight: Frozen
encoder.model.layer1.1.bn2.bias: Frozen
encoder.model.layer1.1.conv3.weight: Frozen
encoder.model.layer1.1.bn3.weight: Frozen
encoder.model.layer1.1.bn3.bias: F

In [3]:
criterion = nn.CrossEntropyLoss()  # Suitable for logits or probabilities
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, custom_model.parameters()), lr=0.001)

In [4]:
print("Parameters being optimized:")
for i, param_group in enumerate(optimizer.param_groups):
    print(f"Parameter group {i}:")
    for param in param_group["params"]:
        print(param.shape)

Parameters being optimized:
Parameter group 0:
torch.Size([64, 64])
torch.Size([64])
torch.Size([15, 64])
torch.Size([15])


In [5]:
cd ..

c:\Users\LDRMMM01\Desktop\contrastive_learning


In [6]:
import json
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

class MvTecDataset(Dataset):
    def __init__(self, json_path='./dataset/dataset.json', train=True, img_size=256,
                 mean=[0.4914, 0.4822, 0.4465],  std=[0.2470, 0.2435, 0.2616]):
        """Loads image paths and labels from a JSON file."""
        with open(json_path, 'r') as file:
            dataset_dict = json.load(file)
        
        self.items = dataset_dict['train'] if train else dataset_dict['val']
        
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip() if train else transforms.RandomApply([]),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = self.items[idx]
        img_path, class_label = item["path"], item["label"]
        
        try:
            img = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image: {img_path} - {e}")
            return None  # Skip problematic images
        
        img = self.transform(img)
        return img, class_label

def create_dataloaders(json_path='./dataset/dataset.json', batch_size=32, img_size=256, num_workers=4):
    """Creates DataLoaders for training and validation datasets."""
    train_dataset = MvTecDataset(json_path=json_path, train=True, img_size=img_size)
    val_dataset = MvTecDataset(json_path=json_path, train=False, img_size=img_size)
    
    # Splitting validation dataset: 70% - 30%
    val_size = int(0.7 * len(val_dataset))
    test_size = len(val_dataset) - val_size
    val_subset, test_subset = random_split(val_dataset, [val_size, test_size])

    # Creating DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    return train_loader, val_loader, test_loader

# Debugging Tip: If you get worker errors, try setting num_workers=0
temp, data_loader_train, data_loader_val = create_dataloaders(num_workers=0)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Hyperparameters
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, custom_model.parameters()), lr=learning_rate)

# Move model to device
custom_model.to(device)

# Training loop with validation
for epoch in range(num_epochs):
    # Training phase
    custom_model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0

    for batch_idx, (inputs, targets) in enumerate(tqdm(data_loader_train, desc=f"Epoch {epoch+1}/{num_epochs} - Training")):
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass
        outputs = custom_model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track metrics
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += targets.size(0)
        correct_train += predicted.eq(targets).sum().item()

    train_acc = 100. * correct_train / total_train
    avg_train_loss = train_loss / len(data_loader_train)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}%")

    # Validation phase
    custom_model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(data_loader_val, desc=f"Epoch {epoch+1}/{num_epochs} - Validation")):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = custom_model(inputs)
            loss = criterion(outputs, targets)

            # Track metrics
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += targets.size(0)
            correct_val += predicted.eq(targets).sum().item()

    val_acc = 100. * correct_val / total_val
    avg_val_loss = val_loss / len(data_loader_val)

    print(f"Epoch {epoch+1}/{num_epochs} - Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

Epoch 1/20 - Training: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]


Epoch 1/20 - Train Loss: 2.7070 | Train Acc: 19.21%


Epoch 1/20 - Validation: 100%|██████████| 9/9 [00:07<00:00,  1.28it/s]


Epoch 1/20 - Val Loss: 2.7055 | Val Acc: 25.27%


Epoch 2/20 - Training: 100%|██████████| 20/20 [00:16<00:00,  1.19it/s]


...

Epoch 19/20 - Val Loss: 2.1737 | Val Acc: 75.09%


Epoch 20/20 - Training: 100%|██████████| 20/20 [00:16<00:00,  1.19it/s]


Epoch 20/20 - Train Loss: 2.1801 | Train Acc: 78.74%


Epoch 20/20 - Validation: 100%|██████████| 9/9 [00:07<00:00,  1.28it/s]

Epoch 20/20 - Val Loss: 2.1534 | Val Acc: 76.19%


In [10]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

# Define the ClassifierHead
class ClassifierHead(nn.Module):
    def __init__(self, base_model, embedding_dim):
        super(ClassifierHead, self).__init__()
        self.encoder = base_model  # Use the base model as the encoder
        self.fc1 = nn.Linear(embedding_dim, 64)  # First FC layer
        self.relu = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(64, 15)  # Classification head for 10 classes
        self.softmax = nn.Softmax(dim=1)  # Softmax for probabilities
    
    def forward(self, x):
        # Forward pass through the encoder
        x = self.encoder(x)
        # Pass through the new fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)  # Apply softmax for probabilities
        return x

# Initialize a randomly initialized ResNet
random_resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Replace the final fully connected layer with an identity function
random_resnet.fc = nn.Identity()

# Freeze all parameters in the ResNet
for param in random_resnet.parameters():
    param.requires_grad = False

# Define the embedding dimension (ResNet-50's output features are 2048)
embedding_dim = 2048

# Instantiate the custom model
custom_model = ClassifierHead(random_resnet, embedding_dim)

# Verify trainable parameters
print("Trainable Parameters:")
for name, param in custom_model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")


Trainable Parameters:
encoder.conv1.weight: Frozen
encoder.bn1.weight: Frozen
encoder.bn1.bias: Frozen
encoder.layer1.0.conv1.weight: Frozen
encoder.layer1.0.bn1.weight: Frozen
encoder.layer1.0.bn1.bias: Frozen
encoder.layer1.0.conv2.weight: Frozen
encoder.layer1.0.bn2.weight: Frozen
encoder.layer1.0.bn2.bias: Frozen
encoder.layer1.0.conv3.weight: Frozen
encoder.layer1.0.bn3.weight: Frozen
encoder.layer1.0.bn3.bias: Frozen
encoder.layer1.0.downsample.0.weight: Frozen
encoder.layer1.0.downsample.1.weight: Frozen
encoder.layer1.0.downsample.1.bias: Frozen
encoder.layer1.1.conv1.weight: Frozen
encoder.layer1.1.bn1.weight: Frozen
encoder.layer1.1.bn1.bias: Frozen
encoder.layer1.1.conv2.weight: Frozen
encoder.layer1.1.bn2.weight: Frozen
encoder.layer1.1.bn2.bias: Frozen
encoder.layer1.1.conv3.weight: Frozen
encoder.layer1.1.bn3.weight: Frozen
encoder.layer1.1.bn3.bias: Frozen
encoder.layer1.2.conv1.weight: Frozen
encoder.layer1.2.bn1.weight: Frozen
encoder.layer1.2.bn1.bias: Frozen
encoder.

In [11]:
import torch
import torch.nn as nn
from torchvision.models import resnet50
from tqdm import tqdm

random_resnet = resnet50(pretrained=False)
random_resnet.fc = nn.Identity()  # Replace the final layer with identity
for param in random_resnet.parameters():
    param.requires_grad = False  # Freeze all layers

embedding_dim = 2048
custom_model = ClassifierHead(random_resnet, embedding_dim)

# Move the model to GPU
custom_model = custom_model.cuda()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, custom_model.parameters()), lr=1e-3)

epochs = 20  # Number of epochs
for epoch in range(epochs):
    custom_model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Wrap the batch iteration with tqdm for progress bar
    with tqdm(data_loader_train, desc=f"Epoch {epoch+1}/{epochs} - Training") as pbar_train:
        for inputs, labels in pbar_train:
            inputs, labels = inputs.cuda(), labels.cuda()  # Move inputs and labels to GPU

            optimizer.zero_grad()  # Clear gradients from previous step

            # Forward pass
            outputs = custom_model(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagate
            
            optimizer.step()  # Update weights

            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar
            pbar_train.set_postfix(loss=running_loss / (pbar_train.n + 1), accuracy=100 * correct / total)

    avg_loss = running_loss / len(data_loader_train)
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Validation loop with tqdm
    custom_model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation for validation
        with tqdm(data_loader_val, desc=f"Epoch {epoch+1}/{epochs} - Validation") as pbar_val:
            for inputs, labels in pbar_val:
                inputs, labels = inputs.cuda(), labels.cuda()  # Move inputs and labels to GPU
                
                outputs = custom_model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Update progress bar
                pbar_val.set_postfix(accuracy=100 * correct / total)

    val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

c:\eng_apps\Anaconda3\envs\simclr\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\eng_apps\Anaconda3\envs\simclr\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/20 - Training: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s, accuracy=13.2, loss=2.66]


Epoch 1/20, Loss: 2.6585, Accuracy: 13.23%


Epoch 1/20 - Validation: 100%|██████████| 9/9 [00:06<00:00,  1.29it/s, accuracy=12.1]


Validation Accuracy: 12.09%


Epoch 2/20 - Training: 100%|██████████| 20/20 [00:16<00:00,  1.20it/s, accuracy=21.1, loss=2.61]


...

Validation Accuracy: 47.62%


Epoch 20/20 - Training: 100%|██████████| 20/20 [00:16<00:00,  1.19it/s, accuracy=39.5, loss=2.43]


Epoch 20/20, Loss: 2.4308, Accuracy: 39.53%


Epoch 20/20 - Validation: 100%|██████████| 9/9 [00:07<00:00,  1.29it/s, accuracy=47.3]

Validation Accuracy: 47.25%
